In [1]:
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
from PIL import Image
import torch
import os 

os.environ["TRANSFORMERS_OFFLINE"] = "1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/zhekai/miniforge3/envs/openvla/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-04 14:26:25.263384: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-04 14:26:25.290570: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-04 14:26:25.290591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-04 14:

In [2]:
# Load Processor & VLA

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
# quantization_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=10.0)
# path = '/cluster/nvme9a/dzk/'
processor = AutoProcessor.from_pretrained("openvla/openvla-7b", trust_remote_code=True)
vla = AutoModelForVision2Seq.from_pretrained(
    "openvla/openvla-7b", 
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2",  # [Optional] Requires `flash_attn`
    low_cpu_mem_usage=True, 
    trust_remote_code=True,
    quantization_config=quantization_config,
)#.to(device)

# Grab image input & format prompt
image = Image.open("test.png")
instruction = "put eggplant into pot"
prompt = f"In: What action should the robot take to {instruction}?\nOut:"

# Predict Action (7-DoF; un-normalize for BridgeData V2)
inputs = processor(prompt, image).to(device, dtype=torch.bfloat16)
action = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)
# Execute...
# robot.act(action, ...)

Expected `transformers==4.40.1` and `tokenizers==0.19.1` but got `transformers==4.46.1` and `tokenizers==0.20.1`; there might be inference-time regressions due to dependency changes. If in doubt, pleaseuse the above versions.
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.11it/s]
We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)


In [3]:
# from torch.profiler import profile, record_function, ProfilerActivity

# === BFLOAT16 MODE ===
inputs = processor(prompt, image).to(device, dtype=torch.bfloat16)
# inputs["input_ids"] = inputs["input_ids"][:, 1:]

# Run OpenVLA Inference
torch.manual_seed(0)

def trace_handler(prof):
    # print(prof.key_averages().table(
    #     sort_by="self_cuda_time_total", row_limit=-1))
    prof.export_chrome_trace("tmp/test_trace_" + str(prof.step_num) + ".json")

# with torch.profiler.profile(
#     activities=[
#         torch.profiler.ProfilerActivity.CPU,
#         torch.profiler.ProfilerActivity.CUDA,
#     ],
#     schedule=torch.profiler.schedule(
#         wait=1,
#         warmup=1,
#         active=1),
#     on_trace_ready=trace_handler,
#     with_stack=True,
#     profile_memory=True,
#     with_flops = True
#     ) as p:
#         for iter in range(3):
#             action = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)
#             p.step()



In [4]:
# profile latecy with cuda event
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
times = []
for i in range(10):
    start.record()
    action = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)
    end.record()
    torch.cuda.synchronize()
    times.append(start.elapsed_time(end))
print("Average inference time: ", sum(times)/len(times))
print("Std: ", torch.tensor(times).std().item())

Average inference time:  184.96997528076173
Std:  2.2852206230163574


In [5]:
#profile memroy with torch
print(torch.cuda.memory_summary(device))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   4478 MiB |   4762 MiB | 225576 MiB | 221098 MiB |
|       from large pool |   4271 MiB |   4555 MiB | 219447 MiB | 215176 MiB |
|       from small pool |    206 MiB |    210 MiB |   6128 MiB |   5921 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   4478 MiB |   4762 MiB | 225576 MiB | 221098 MiB |
|       from large pool |   4271 MiB |   4555 MiB | 219447 MiB |